Fit KNNRecommender and save the resulting Meta-Table and Model

In [ ]:
from knn_recommender import KNNRecommender

knn_recommender = KNNRecommender(supervised=True, _load=False)
knn_recommender.fit(complete_data_path="./datasets/", train_data_path="./data/train_data/", test_data_path="./data/test_data/")

knn_recommender.save_meta_table("./recommender_data/knn_meta_table.h5")
knn_recommender.persist_model("./recommender_data/KNNRecommender.joblib")

Fit RegressionRecommender and save the resulting Meta-Table and Model

In [ ]:
from regression_recommender import RegressionRecommender

recommender = RegressionRecommender(supervised=True, _load=False)
# recommender.fit(complete_data_path="./datasets/", train_data_path="./data/train_data/", test_data_path="./data/test_data/")

# recommender.save_meta_table("./recommender_data/regression_meta_table.h5")
# recommender.persist_model("./recommender_data/RegressionRecommender.joblib")

Evaluate QuantifierRecommender with Leave-One-Out

In [ ]:
from regression_recommender import RegressionRecommender

recommender = RegressionRecommender(supervised=True)
# recommender.load_fit_meta_table("./recommender_data/meta_table.h5")
recommender_eval, quantifiers_eval = recommender.leave_one_out_evaluation("./plot_data/recommender_evaluation_table.csv",
                                                                          "./plot_data/quantifiers_evaluation_table.csv")

Evaluate the KQuantifier for each dataset using APP

In [ ]:
from k_quantifier import KQuantifier

k_quantifier = KQuantifier()
k_quantifier_eval = k_quantifier.evaluation(recommender_eval, quantifiers_eval, "./plot_data/k_quantifier_evaluation_table.csv")

Join the Quantifiers Evaluation Table with the KQuantifier Evaluation Table

In [ ]:
import pandas as pd

complete_evaluation_table = pd.concat([quantifiers_eval, k_quantifier_eval], axis=0)
complete_evaluation_table.to_csv("./plot_data/complete_evaluation_table.csv", index=False)

Example of using the KQuantifier

In [ ]:
from k_quantifier import KQuantifier
import pandas as pd
import numpy as np
from utils_ import load_train_test_data
from sklearn.preprocessing import MinMaxScaler
import pdb

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

dataset = pd.read_csv("./datasets/AedesQuinx.csv")
scaler = MinMaxScaler()

y = dataset.pop(dataset.columns[-1])
y = y.to_numpy()
X = dataset.to_numpy()
# X = scaler.fit_transform(dataset)
# X = np.c_[X, y]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train2, y_train2, X_test2, y_test2 = load_train_test_data(dataset_name="AedesQuinx",
                                                            train_data_path="./data/train_data",
                                                            test_data_path="./data/test_data")


cancer = load_breast_cancer()

# Features (X) and labels (y)
X = cancer.data
y = cancer.target

# Split the dataset into training and test sets
# test_size=0.2 means 20% of data will go to the test set, 80% to the train set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# k_quantifier_median = KQuantifier(k=-1, method="median")
k_quantifier_weighted = KQuantifier(k=-1, method="weighted")

# k_quantifier_median.fit(X_train2, y_train2)
k_quantifier_weighted.fit(X_train2, y_train2)


In [ ]:
ranking = k_quantifier_weighted.predict(X_test2)

In [ ]:
ranking

In [ ]:
error_list = recommender_eval.loc['AedesQuinx']['predicted_ranking_error']
k = 3
error_list

weight_list = [0.5, 0.3, 0.2]
k_prev = [0.3, 0.26, 0.35]

final_k_prev = np.sum(np.array(k_prev) * np.array(weight_list))

In [ ]:
print(0.5 * 0.3 + 0.3 * 0.26 + 0.2 * 0.35)

In [ ]:
final_k_prev

In [ ]:
print(f"MEDIAN: {k_quantifier_median.predict(X_test2)}\nWEIGHTED: {k_quantifier_weighted.predict(X_test2)}\nTRUE: {sum(y_test2)/len(y_test2)}")

In [ ]:
import pandas as pd

k_eval = pd.read_csv("./plot_data/k_quantifier_evaluation_table.csv")
k_eval = k_eval[k_eval['dataset'] == 'AedesQuinx']
k_eval = k_eval[k_eval['alpha'] == 0.11]
k_eval